# Kevin de León district election results analysis

By [Ben Welsh](https://palewi.re/who-is-ben-welsh/)

A November 2018 contest for California's U.S. Senate seat matching incumbent Sen. Dianne Feinstein against former state Sen. Kevin de León. This analysis investigates how de León fared in his own district. It was conducted for the February 11, 2019, Los Angeles Times story ["Former state Sen. Kevin de León will run to replace Jose Huizar on L.A. City Council."](https://www.latimes.com/local/lanow/la-me-ln-de-leon-city-council-20190211-story.html)

### Import Python tools

In [2]:
import pandas as pd
import geopandas as gpd

In [34]:
import warnings
warnings.simplefilter("ignore")

### Analyze election results

Election results published in PDF form by the [Los Angeles County Registrar/Recorder](https://lavote.net/home/voting-elections/current-elections/election-results/past-election-results#11062018) are available as comma-separate values thanks to the [OpenElections](http://openelections.net/) project.

In [8]:
results = pd.read_csv('https://raw.githubusercontent.com/openelections/openelections-data-ca/master/2018/20181106__ca__general__los_angeles__precinct.csv')

The precinct numbers need to be split off from city names.

In [5]:
parse_precinct_number = lambda val: val.split(" ")[0].strip()

In [9]:
results['precinct_number'] = results.precinct.apply(parse_precinct_number)

Filter down to results from the U.S. Senate contest

In [10]:
us_senate = results[results.office == 'US Senate']

Filter down to results from de León's constituency, the 24th State Senate district.

In [11]:
senate24 = df[
    (df.office == 'State Senate') &
    (df.district == 24)
]

Filter the U.S. Senate results down to de León district.

In [12]:
filtered = us_senate[us_senate.precinct_number.isin(senate24.precinct_number)]

Clean up the columns.

In [13]:
trimmed = filtered[[
    'precinct_number',
    'candidate',
    'votes'
]].rename(columns={"precinct_number": "precinct"})

Calculate the total votes for each candidate.

In [17]:
trimmed.groupby("candidate").votes.sum().reset_index()

,candidate,votes
0,Dianne Feinstein,111860
1,Kevin DeLeon,105236


Calculate the difference.

In [19]:
111860 - 105236

6624

Convert the precint results to a pivot table.

In [22]:
pivot = trimmed.pivot(index="precinct", columns="candidate", values="votes").reset_index()

Clean up the column headers.

In [23]:
trimmed_pivot = pivot[[
    'precinct',
    'Dianne Feinstein',
    'Kevin DeLeon'
]].rename(columns={
    'Dianne Feinstein': "feinstein",
    'Kevin DeLeon': "deleon"
})

Add a column with the winner

In [25]:
def get_winner(row):
    if row.feinstein == row.deleon:
        return 'tie'
    elif row.feinstein > row.deleon:
        return 'feinstein'
    else:
        return 'deleon'

In [26]:
trimmed_pivot['winner'] = trimmed_pivot.apply(get_winner, axis=1)

Add up how many precincts each candidate won.

In [27]:
trimmed_pivot.winner.value_counts()

feinstein    164
deleon       151
tie            4
Name: winner, dtype: int64

Calculate the margin of victory in each precinct.

In [45]:
get_margin = lambda row: abs(row.feinstein - row.deleon)

In [47]:
trimmed_pivot['margin'] = trimmed_pivot.apply(get_margin, axis=1)

Write out the pivot to a CSV for inspection.

In [56]:
trimmed_pivot.to_csv("./output/sd-24-results.csv", index=False)

### Merge results to map

Read in a shapefile of precincts from the Los Angeles County Registrar/Recorder.

In [49]:
shp = gpd.read_file("./input/RRCC_PRECINCTS_ELECTION.shp")

Clean up the columns.

In [50]:
trimmed_shp = shp[[
    'Precinct',
    'geometry'
]].rename(columns={"Precinct": "precinct"})

Correct a couple bad precinct numbers discovered by visual inspection.

In [51]:
def correct_shp(precinct):
    if precinct == '9001668B':
        return '9001668A'
    elif precinct == '9005012B':
        return '9005012A'
    else:
        return precinct

In [52]:
trimmed_shp['precinct'] = trimmed_shp.precinct.apply(correct_shp)

Merge the shapefile to the results

In [53]:
merged_shp = trimmed_shp.merge(trimmed_pivot, on="precinct", how="inner")

Write out the result.

In [60]:
merged_shp.to_crs({"init": "EPSG:4236"}).to_file("./output/sd-24-polygons.shp")

Convert polygons to points

In [58]:
merged_shp['geometry'] = merged_shp.geometry.centroid

Write that out.

In [59]:
merged_shp.to_crs({"init": "EPSG:4236"}).to_file("./output/sd-24-points.shp")

### Map results in QGIS

The [map.qgs](./map.qgs) file included with this repository plots the results. Circles are sized by margin of victory. Yellow precincts were won by Feinstein. Green precincts were won by de León.

<img src="./output/screenshot.png">